In [1]:
import json
import time
from os import listdir
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
## ABRIR O ULTIMO DB

megasena_db_dir = './db/megasena/'
dbs = listdir(megasena_db_dir)
filename = megasena_db_dir+dbs[-1]
with open(filename, 'r') as openfile: 
    db = json.load(openfile)

In [3]:
## Selenium Configuration and actuation ##

opts = Options()
opts.add_argument(f"user-agent=[Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36]")

browser = webdriver.Chrome()

browser.get("http://loterias.caixa.gov.br/wps/portal/loterias/landing/megasena/") 
ultimo_concurso = browser.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span').text
ultimo_concurso = (ultimo_concurso.split())[1]


# O formato da chave é "Concurso NNN (dd/mm/aaaa)"
def find_concurso(dica, db):
    found_key = None
    for key in db.keys():
        if str(dica) in key:
            found_key = key
    if found_key == None:
        found_key = "Concurso não encontrado."
    return found_key
    
def find_buraco(db, ultimo_concurso):
    concurso = 1
    faltantes = []
    while concurso <= int(ultimo_concurso):
        if find_concurso('Concurso '+str(concurso), db) in 'Concurso não encontrado.':
            faltantes.append(concurso)
        concurso+=1
    return faltantes


faltantes = find_buraco(db, ultimo_concurso)
print('\nNão foram encontrados os concursos:', faltantes, '\n\nUm total de:', len(faltantes))


resultados = {}

for faltante in faltantes:
    busca = browser.find_element_by_xpath('//*[@id="buscaConcurso"]')
    busca.send_keys(Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE)
    busca.send_keys(faltante)
    if ' '+str(faltante)+' ' not in (browser.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span')).text:
        busca.send_keys(Keys.ENTER)
        time.sleep(1)
    concurso = (browser.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span')).text
    numeros = []
    numeros.append((browser.find_element_by_xpath('//*[@id="ulDezenas"]')).text)
    resultado_numeros = []
    while len(numeros[0]) > 0:
        resultado_numeros.append(int(numeros[0][:2]))
        numeros[0] = numeros[0][2:]
    numeros = resultado_numeros
    resultados[concurso] = {
        'Numeros': numeros
    }
    print(concurso, numeros)
browser.quit()
print('Done.')

## ## ##


Não foram encontrados os concursos: [2258, 2259, 2260] 

Um total de: 3
Concurso 2258 (05/05/2020) [1, 5, 7, 14, 23, 26]
Concurso 2259 (07/05/2020) [20, 27, 41, 54, 56, 58]
Concurso 2260 (09/05/2020) [12, 14, 34, 35, 37, 47]
Done.


In [4]:
## SALVAR NOVO DB
db.update(resultados)

megasena_db_dir = './db/megasena/megasena_'
dictionary = db
filename = megasena_db_dir+str(int(time.time()))+'.json'
with open(filename, "w") as outfile: 
    json.dump(dictionary, outfile)